In [1]:
!pip install -q PyArabic

     |████████████████████████████████| 126 kB 7.7 MB/s 


In [2]:
import tensorflow as tf
import numpy as np
import os
import time
import glob
import pandas as pd
from random import shuffle
from tensorflow.keras.layers import GRU, Embedding, Dense, Input, Dropout, Bidirectional, BatchNormalization, Flatten, Reshape
from tensorflow.keras.models import Sequential
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pyarabic.araby as araby
import re

In [3]:
metadata_path = '/content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/metadata.xlsx'
test_metadata_path = '/content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/test_metadata.xlsx'

In [4]:
def preprocess_metadata(metadata):
  metadata['Bahr'] = metadata['Bahr'].apply(lambda bahr: bahr.strip() if isinstance(bahr,str) else bahr)
  metadata['Clean Script'] = metadata['Script'].apply(lambda script: araby.strip_tashkeel(araby.strip_tatweel(script)))
  return metadata

In [5]:
metadata = preprocess_metadata(pd.read_excel(metadata_path))
test_metadata = preprocess_metadata(pd.read_excel(test_metadata_path))
train_metadata = metadata[[script not in test_metadata['No.'].values for script in metadata['No.']]]
train_metadata

,No.,Utterance name,Script,Bahr,Source,Name when extacted,Original name in document,Is_Malformed,Reciter_Label,Reciter_Gender,Further info on Bahr,Name of document,Clean Script
0,1.0,P-1.aac,شتَّ شَعبُ الحيِّ بعدَ التئامْ وشجاكَ الر...,المديد,ديوان الطرماح ص227,1.aac,1.aac,0.0,0.0,0.0,NaN,بحر المديد,شت شعب الحي بعد التئام وشجاك الربع ربع ال...
1,2.0,P-2.aac,حَسَرَتْ عَنْهُ الرِّيَاحُ فَأَبْدَتْ من...,المديد,ديوان الطرماح ص227,2.aac,2.aac,0.0,0.0,0.0,NaN,NaN,حسرت عنه الرياح فأبدت منتأى كالقرو رهن ا...
2,3.0,P-3.aac,وخصيفَ اللَّونِ جادَتْ بهِ مَرْخَةٌ مِنْ ...,المديد,ديوان الطرماح ص227,3.aac,3.aac,0.0,0.0,0.0,NaN,NaN,وخصيف اللون جادت به مرخة من مخدج أو تمام
3,4.0,P-4.aac,بَيْنَ أظْآرٍ بِمَظْلُومَةٍ كَسَرَاةِ الس...,المديد,ديوان الطرماح ص227,4.aac,4.aac,0.0,0.0,0.0,NaN,NaN,بين أظآر بمظلومة كسراة الساق ساق الحمام
4,5.0,P-5.aac,مَنْزِلًا كَانَ لَنَا مَرَّةً وطنًا نحتلّ...,المديد,ديوان الطرماح ص227,5.aac,5.aac,0.0,0.0,0.0,NaN,NaN,منزلا كان لنا مرة وطنا نحتله كل عام
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3710,3760.0,P-3760.ogg,فَظَلِلْتُ بَعْدَ وَفَاتِهِ مُتَبَلِّداً ...,الكامل,ديوان حسان,WhatsApp Audio 2019-11-30 at 3.01.58 PM(2).ogg,WhatsApp Audio 2019-11-30 at 3.01.58 PM(2).ogg,NaN,NaN,NaN,NaN,NaN,فظللت بعد وفاته متبلدا يالهف نفسي ليتني ل...
3711,3761.0,P-3761.ogg,أوْ حلّ أمرُ اللهِ فينا عاجلاً في روحة ٍ ...,الكامل,ديوان حسان,WhatsApp Audio 2019-11-30 at 3.01.58 PM(3).ogg,WhatsApp Audio 2019-11-30 at 3.01.58 PM(3).ogg,NaN,NaN,NaN,NaN,NaN,أو حل أمر الله فينا عاجلا في روحة من يوم...
3712,3762.0,P-3762.ogg,يَا بِكْرَ آمِنَة َ المُبَارَكَ ذِكْرُهُ ...,الكامل,ديوان حسان,WhatsApp Audio 2019-11-30 at 3.01.59 PM(1).ogg,WhatsApp Audio 2019-11-30 at 3.01.59 PM(1).ogg,NaN,NaN,NaN,NaN,NaN,يا بكر آمنة المبارك ذكره ولدتك محصنة بس...
3713,3763.0,P-3763.ogg,نُوراً أضَاءَ على البَرِيّة ِ كُلّها مَنْ...,الكامل,ديوان حسان,WhatsApp Audio 2019-11-30 at 3.01.59 PM(2).ogg,WhatsApp Audio 2019-11-30 at 3.01.59 PM(2).ogg,NaN,NaN,NaN,NaN,NaN,نورا أضاء على البرية كلها من يهد للنور ا...


In [6]:
X = list(train_metadata['Clean Script'])
y = list(train_metadata['Bahr'])
X_test = list(test_metadata['Clean Script'])
y_test = list(test_metadata['Bahr'])

In [7]:
bahrs = sorted(set(y))
bahrs

['البسيط',
 'الخفيف',
 'الرجز',
 'الرمل',
 'السريع',
 'الطويل',
 'الكامل',
 'المتدارك',
 'المتقارب',
 'المجتث',
 'المديد',
 'المضارع',
 'المقتضب',
 'المنسرح',
 'الهزج',
 'الوافر']

In [8]:
labels = [index for index,_ in enumerate(bahrs)]
name2label = lambda name: bahrs.index(name)
print(name2label('الوافر'))
label2name = lambda label: bahrs[label]
print(label2name(15))
labels

15
الوافر


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [9]:
X_train, X_valid , y_train, y_valid = train_test_split(X, y, test_size = 0.15, random_state = 41)

In [10]:
vocab = list('إةابتثجحخدذرزسشصضطظعغفقكلمنهويىأءئؤ#آ ')

In [11]:
# Creating a mapping from unique characters to indices
char2idx = {u:i+1 for i, u in enumerate(vocab)}

def to_sequences(X):
  # X = [re.sub(' +',' ',x) for x in X]
  X = [[char2idx[char] for char in x if char in vocab] for x in X]
  X = pad_sequences(X, padding='post', value=0, maxlen = 128)
  return X

In [12]:
X_train = to_sequences(X_train)
X_valid = to_sequences(X_valid)
X_test  = to_sequences(X_test)

y_train = np.array([name2label(i) for i in y_train])
y_valid = np.array([name2label(i) for i in y_valid])
y_test = np.array([name2label(i) for i in y_test])

In [13]:
X_train[0]

array([29, 22,  3, 33, 38, 29, 22,  3, 33,  5, 38,  4,  3, 10, 12,  3,  5,
       38, 29, 24, 25, 28,  3, 38, 38, 38, 38, 38, 38, 20, 25, 31, 38, 27,
       24, 19, 38, 26, 26,  3, 38, 30, 24,  3,  5, 26, 38, 26,  7, 26, 25,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32)

In [14]:
y_test[0]

10

In [15]:
def create_model_v1():
  model = Sequential()
  model.add(Input((128,)))
  model.add(Embedding(len(char2idx)+1, 32))
  model.add(Bidirectional(GRU(units = 64, return_sequences=True)))
  model.add(Bidirectional(GRU(units = 64, return_sequences=True)))
  model.add(Bidirectional(GRU(units = 64, return_sequences=True)))
  model.add(Bidirectional(GRU(units = 64)))
  model.add(Dense(len(bahrs), activation = 'softmax'))
  model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  return model

In [16]:
checkpoint_path = "./cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_delta=0.0001, min_lr=0.0001)]
callbacks += [tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 monitor='val_accuracy',
                                                 verbose=1,
                                                 save_weights_only=True,
                                                 save_best_only= True,
                                                 mode='max')]

In [17]:
model = create_model_v1()

In [18]:
model.fit(X_train, y_train, validation_data= (X_valid, y_valid), epochs = 15, batch_size=256, shuffle = True, callbacks=callbacks)

Epoch 1/15
11/12 [==========================>...] - ETA: 0s - loss: 2.6013 - accuracy: 0.2660
Epoch 1: val_accuracy improved from -inf to 0.32738, saving model to ./cp.ckpt
12/12 [==============================] - 21s 358ms/step - loss: 2.6040 - accuracy: 0.2644 - val_loss: 2.3140 - val_accuracy: 0.3274 - lr: 0.0010
Epoch 2/15
11/12 [==========================>...] - ETA: 0s - loss: 2.4262 - accuracy: 0.2848
Epoch 2: val_accuracy did not improve from 0.32738
12/12 [==============================] - 1s 102ms/step - loss: 2.4259 - accuracy: 0.2847 - val_loss: 2.3070 - val_accuracy: 0.3274 - lr: 0.0010
Epoch 3/15
11/12 [==========================>...] - ETA: 0s - loss: 2.4062 - accuracy: 0.2855
Epoch 3: val_accuracy did not improve from 0.32738
12/12 [==============================] - 1s 102ms/step - loss: 2.4079 - accuracy: 0.2847 - val_loss: 2.3166 - val_accuracy: 0.3274 - lr: 0.0010
Epoch 4/15
11/12 [==========================>...] - ETA: 0s - loss: 2.4055 - accuracy: 0.2852
Epoch 4: v

In [19]:
model = create_model_v1()
model.load_weights(checkpoint_path)
model.evaluate(X_test, y_test, batch_size = 256)

2/2 [==============================] - 5s 27ms/step - loss: 2.2849 - accuracy: 0.2535


[2.284884452819824, 0.2534818947315216]

In [20]:
# def classify(sentence):
# #   sentence = process_review(sentence)
#   cleaned_bait = ""
#   for char in sentence:
#       if char in vocab:
#         cleaned_bait += char
#   sequence = [char2idx[char] for char in cleaned_bait]
#   sequence = pad_sequences([sequence], maxlen = X_train.shape[1], padding='post', value=0)

#   pred = model.predict(sequence)[0]
#   print(label2name[np.argmax(pred, 0).astype('int')], np.max(pred))
  

In [21]:
# import seaborn as sn
# import pandas as pd
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix

# cm = confusion_matrix(y_test, y_pred.argmax(-1))
# cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
# df_cm = pd.DataFrame(cmn, labels, labels)
# plt.figure(figsize=(10,7))
# sn.heatmap(df_cm, annot=True, annot_kws={"size": 10}, fmt='.2f') # font size

# plt.show()